# Imports

In [1]:
import os
import re
import time
import pickle

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

from tqdm.notebook import tqdm

import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from torch.optim.lr_scheduler import ExponentialLR

from util_func import sMAPE

# Config enviroment

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

# Prepare data

In [3]:
def read_hec(data_root, force_reload=False):
    try:
        with open(data_root+"loaded_dataset.pk", 'rb') as f:
            df = pickle.load(f)
    except:
        force_reload = True

    if force_reload:
        df = pd.DataFrame({"MW":[], 'hour':[], 'dayofweek':[], 'month':[], 'dayofyear':[], 'state':[]})
        
        for file in tqdm(os.listdir(data_root)):
            if file[-4:] != ".csv" or file == "pjm_hourly_est.csv":
                continue
            
            state = re.split('_', file)[0]

            temp_df = pd.read_csv('{}/{}'.format(data_root, file), parse_dates=[0])
            temp_df['hour'] = temp_df.apply(lambda x: x['Datetime'].hour,axis=1)
            temp_df['dayofweek'] = temp_df.apply(lambda x: x['Datetime'].dayofweek,axis=1)
            temp_df['month'] = temp_df.apply(lambda x: x['Datetime'].month,axis=1)
            temp_df['dayofyear'] = temp_df.apply(lambda x: x['Datetime'].dayofyear,axis=1)
            temp_df["state"] = state
            temp_df = temp_df.rename(columns={"{}_MW".format(state): "MW"})
            temp_df = temp_df.sort_values("Datetime").drop("Datetime",axis=1).reset_index(drop=True)
            
            df = df.append(temp_df, ignore_index=True)

        with open(data_root+"loaded_dataset.pk", 'wb') as f:
            pickle.dump(df, f)

    return df

def preprocess_hec(df, train_prop, look_back):
    df = df.copy()
    train_size = int(np.ceil(df.shape[0] * train_prop))
    sc = MinMaxScaler()
    label_sc = MinMaxScaler()
    
    label_sc.fit(df["MW"].values.reshape(-1,1))
    features = ["MW", "hour", "dayofweek", "month", "dayofyear"]
    df[features] = sc.fit_transform(df[features])
    one_hot = pd.get_dummies(df["state"])
    states = df["state"]
    df.drop("state", axis=1, inplace=True)
    df = df.join(one_hot)
    
    data = df.to_numpy()

    inputs = []
    labels = []

    for i in tqdm(range(look_back, len(data))):
        if states[i-look_back:i].nunique() == 1:     
            inputs.append(data[i-look_back:i])
            labels.append(data[i,0])

    inputs = np.array(inputs)
    labels = np.array(labels).reshape(-1,1)

    X_train = inputs[:train_size]
    y_train = labels[:train_size]

    X_test = inputs[train_size:]
    y_test = labels[train_size:]

    return (X_train, y_train), (X_test, y_test), label_sc

In [4]:
train_prop = 0.9
look_back = 90
data_root = "../Datasets/Time_Series_Datasets/Hourly Energy Consumption/"
df = read_hec(data_root, force_reload=True)
(X_train, y_train), (X_test, y_test) = preprocess_hec(df, train_prop, look_back)

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/1090077 [00:00<?, ?it/s]

ValueError: too many values to unpack (expected 2)

# Models

In [ ]:
class RNN(nn.Module):
    def __init__(self, hidden_dim, output_dim, n_layers, n_static) -> None:
        super().__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.n_static = n_static

        self.rnn = None
        self.fc1 = nn.Linear(hidden_dim, hidden_dim//4)
        self.fc2 = nn.Linear(hidden_dim//4 + n_static, output_dim)
        self.relu = nn.ReLU()
        
    def forward(self, x, h):
        out, h = self.rnn(x[:,:,:-self.n_static], h)
        out = self.fc1(self.relu(out[:,-1]))
        out = self.fc2(torch.cat((out, x[:,-1,-self.n_static:]), 1))
        return out, h
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device)
        return hidden

class GRUNet(RNN):
    '''GRU'''
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, n_static, drop_prob=0.2):
        super(GRUNet, self).__init__(hidden_dim, output_dim, n_layers, n_static)
        self.rnn = nn.GRU(input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)
        

class LSTMNet(RNN):
    '''LSTM'''
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, n_static, drop_prob=0.2):
        super(LSTMNet, self).__init__(hidden_dim, output_dim, n_layers, n_static)
        self.rnn = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)

# Train and Evaluate

In [81]:
def train(model, train_loader, optimizer, criterion):
    model_type = model.__doc__ 
    model_device = 'cuda' if next(model.parameters()).is_cuda else 'cpu'

    model.train()
    
    start_time = time.process_time()
    h = model.init_hidden(batch_size)
    total_loss = 0
    counter = 0
    for x, y in train_loader:
        counter += 1
        if model_type == "GRU":
            h = h.data
        else:
            h = tuple([e.data for e in h])
        model.zero_grad()
        
        out, h = model(x.to(model_device).float(), h)
        loss = criterion(out, y.to(model_device).float())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        if counter%1000 == 0:
            print("Epoch {}......Step: {}/{}....... Average Loss for Epoch: {}".format(epoch, counter, len(train_loader), total_loss/counter))
    current_time = time.process_time()
    print("Epoch {}/{} Done, Average Loss: {}".format(epoch, epochs, total_loss/len(train_loader)))
    print("Time Elapsed for Epoch: {} seconds".format(str(current_time-start_time)))
        
def evaluate(model, test_loader, criterion, label_sc):
    model_type = model.__doc__
    num_batches = len(test_loader)
    model_device = 'cuda' if next(model.parameters()).is_cuda else 'cpu'
    
    model.eval()
    test_loss = 0
    h = model.init_hidden(batch_size)
    predicted_values, targets = np.empty((0,1)), np.empty((0,1))
    with torch.no_grad():
        for X, y in test_loader:
            if model_type == "GRU":
                h = h.data
            else:
              h = tuple([e.data for e in h])

            out, h = model(X.float().to(model_device), h)
            test_loss += criterion(out, y.to(model_device)).item()
            predicted_values = np.concatenate((predicted_values, out.cpu().detach().numpy().reshape(-1,1)))
            targets = np.concatenate((targets, y.numpy()))
        
    test_loss /= num_batches
    predicted_values = label_sc.inverse_transform(predicted_values)
    targets = label_sc.inverse_transform(targets)
    
    print(f"Test results: \n sMAPE: {sMAPE(predicted_values, targets):>0.2f}% \
                          \n Avg loss: {test_loss:>8f} \n")

# Main

In [ ]:
# Constants
data_root = "../Datasets/Time_Series_Datasets/Hourly Energy Consumption/"
train_prop = 0.9
look_back = 90
batch_size = 1024
hidden_dim = 256
input_dim = 5
output_dim = 1
n_layers = 2
n_static = 12
epochs = 5
lr = 0.001

# Read Dataset
df = read_hec(data_root)
(X_train, y_train), (X_test, y_test), label_sc = preprocess_hec(df, train_prop, look_back)

# Create Datasets
train_dataset = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
test_dataset = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))

# Create Dataloaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

# Set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("==> Use accelerator: ", device)

In [77]:
# model = torch.load("../trained_models/hec/trained_gru.model")
model = GRUNet(input_dim, hidden_dim, output_dim, n_layers, n_static)

In [ ]:
model.to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr)
scheduler = ExponentialLR(optimizer, gamma=0.9)

print("==> Start training ...")
print("Training of {} model".format(model.__doc__))

for epoch in range(1,epochs+1):
    print(f"Epoch {epoch}")
    train(model, train_loader, optimizer, criterion)
    evaluate(model, test_loader, criterion, label_sc)

    # Update learning rate
    if epoch % 5 == 0:
        scheduler.step() 

print("Task done!")

In [83]:
torch.save(model, '../trained_models/hec/trained_gru.model')
# torch.save(model, '../trained_models/hec/trained_lstm.model')